In [1]:
import os
import random
import shutil
import pydicom
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy import ndimage
from datetime import date, datetime
from tensorflow import keras

In [2]:
path = os.path.join(os.getcwd(), "報告")
in_path = os.path.join(path, "CT 報告.csv")
out_path = os.path.join(path, "新 CT 報告.csv")

In [3]:
df = pd.read_csv(in_path)
df

,report_uid,report_text,report_html,report_image,uploaded,history,finish_date,finish_time,upload_text,his2wkl_assign,PROC_FLAG,assign,CHECK1,VER1
0,2019.1220.0001050582.350000019086.20200710.095...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001050582 ...,NaN,NaN,1,0,20200710,103723,NaN,N,N,N,NaN,2.7.2020.0415B
1,2019.1220.0002020073.350000019094.20200710.111...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0002020073 ...,NaN,NaN,1,0,20200710,134333,NaN,N,N,N,NaN,2.7.2020.0415B
2,2019.1220.0001391506.350000019130.20200710.192...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001391506 ...,NaN,NaN,1,1,20200711,52244,NaN,N,N,N,NaN,2.6.2020.0302B
3,2019.1220.0001221365.350000019114.20200710.153...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001221365 ...,NaN,NaN,1,0,20200711,52337,NaN,N,N,N,NaN,2.6.2020.0302B
4,2019.1220.0001019088.350000019113.20200710.150...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001019088 ...,NaN,NaN,1,0,20200711,52638,NaN,N,N,N,NaN,2.6.2020.0302B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6273,2019.1220.0000859312.350000025614.20201004.223...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0000859312 ...,NaN,NaN,1,0,20201005,131951,NaN,N,N,N,NaN,2.8.2020.0430B
6274,2019.1220.0002028291.350000025642.20201005.123...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0002028291 ...,NaN,NaN,1,0,20201005,132331,NaN,N,N,N,NaN,2.8.2020.0430B
6275,2019.1220.0002013537.350000025609.20201004.211...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0002013537 ...,NaN,NaN,1,0,20201005,132355,NaN,N,N,N,NaN,2.8.2020.0430B
6276,2019.1220.0001109370.350000025647.20201005.130...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001109370 ...,NaN,NaN,1,0,20201005,132451,NaN,N,N,N,NaN,2.8.2020.0430B


In [4]:
df = df[["report_uid", "report_text"]]
df

,report_uid,report_text
0,2019.1220.0001050582.350000019086.20200710.095...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001050582 ...
1,2019.1220.0002020073.350000019094.20200710.111...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0002020073 ...
2,2019.1220.0001391506.350000019130.20200710.192...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001391506 ...
3,2019.1220.0001221365.350000019114.20200710.153...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001221365 ...
4,2019.1220.0001019088.350000019113.20200710.150...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001019088 ...
...,...,...
6273,2019.1220.0000859312.350000025614.20201004.223...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0000859312 ...
6274,2019.1220.0002028291.350000025642.20201005.123...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0002028291 ...
6275,2019.1220.0002013537.350000025609.20201004.211...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0002013537 ...
6276,2019.1220.0001109370.350000025647.20201005.130...,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001109370 ...


In [5]:
df["report_uid"] = df["report_uid"].str.split(".", expand = True).loc[:, 3]
df

C:\Users\John He\.conda\envs\tf_gpu2.3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,report_uid,report_text
0,350000019086,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001050582 ...
1,350000019094,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0002020073 ...
2,350000019130,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001391506 ...
3,350000019114,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001221365 ...
4,350000019113,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001019088 ...
...,...,...
6273,350000025614,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0000859312 ...
6274,350000025642,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0002028291 ...
6275,350000025609,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0002013537 ...
6276,350000025647,衛生福利部臺北醫院放射診斷科 CT報告\r\n病歷號碼 : 0001109370 ...


In [6]:
exec(open(os.path.join(path, "encrypt.py"), encoding = 'utf-8').read())

for _ in range(len(df)):
    code = df["report_text"][_][df["report_text"][_].find("病歷號碼 :") + 7: df["report_text"][_].find("科    別") - 7]
    df["report_text"][_] = \
    df["report_text"][_].replace(code, encrypt(code)).replace(\
    df["report_text"][_][df["report_text"][_].find("病患姓名 :") + 7: df["report_text"][_].find("性    別") - 7], \
    "NA")


C:\Users\John He\.conda\envs\tf_gpu2.3\lib\site-packages\IPython\core\interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df["report_text"][0]

'衛生福利部臺北醫院放射診斷科  CT報告\r\n病歷號碼 : 9044509683586       科    別 : 急診醫學科       檢查日期 : 20200710\r\n病患姓名 : NA       性    別 : 女       檢查時間 : 095514\r\n年    齡 : 60       醫囑醫師 : 曲新蘭\r\nHead Brain C-\r\nC.C. left abd pain,left abd pain,left abd pain,left shouldr \r\ncontusion,left shouldr contusion,left shouldr contusion, , , \r\n,L/W,L/W,L/W,S：由EMT推床入急診，主訴機車騎士車禍 與汽車 左肩鈍傷、左下腹\r\n痛、頭部左側、右手腕,S：由EMT推床入急診，主訴機車騎士車禍 與汽車 左肩鈍傷\r\n、左下腹痛、頭部左側、右手腕,S：由EMT推床入急診，主訴機車騎士車禍 與汽車 \r\n左肩鈍傷、左下腹痛、頭部左側、右手腕,  P.E OR LAB GCS  E 4\u3000V 5\u3000M 6\u3000\r\nPUPIL  L  \u3000R  \u3000LMP  \u3000 \u3000,GCS  E 4\u3000V 5\u3000M 6\u3000PUPIL  L  \u3000R  \u3000LMP  \u3000 \r\n\u3000,GCS  E 4\u3000V 5\u3000M 6\u3000PUPIL  L  \u3000R  \u3000LMP  \u3000 \u3000,GCS  E 4\u3000V 5\u3000M 6\u3000\r\nPUPIL  L  \u3000R  \u3000LMP  \u3000 \u3000,體溫 37\u3000脈搏 100\u3000呼吸 18\u3000血壓 189/87\u3000身高 \r\n0\u3000體重 0SPO2  ,體溫 37\u3000脈搏 100\u3000呼吸 18\u3000血壓 189/87\u3000身高 0\u3000體重 \r\n0SPO2  ,體溫 37\u3000脈搏 100\u3000呼吸 18\u3000血壓 189/87\u3

In [8]:
df.to_csv(out_path, index = 0)